## Determine Optimal Portfolio Allocation Using Monte Carlo

In [80]:
import datetime as dt
import yfinance as yf
import numpy as np
import pandas as pd
from functools import reduce
from random import randint

# 10 Companies to Design Portfolio Around
tickers = ["coke", 'googl', 'hd', 'ford', 'smg', 'jnj', 'ag', "aapl", "a", "abc", "ba"]
start = "2010-01-01"
end = "2020-01-01"

# Grab yfinance Data
df_portfolio = yf.download(tickers, start, end).reset_index()

[*********************100%***********************]  11 of 11 completed


### Initial Parameters

In [81]:
iterations = 10000

In [82]:
# Calculate Log Returns
df_stock_data = df_portfolio.set_index("Date").copy(deep=True)
df_stock_data = df_stock_data["Adj Close"].dropna()

companies = df_stock_data.columns
company_returns = []

for company in companies:
    df_stock_data[f"{company}".lower()] = np.log(df_stock_data[company] / df_stock_data[company].shift(1))
    company_returns.append(f"{company}".lower())

# Drop Null Returns
df_stock_data.dropna(inplace=True)

df_returns = df_stock_data[company_returns].copy(deep=True)
df_returns

,a,aapl,abc,ag,ba,coke,ford,googl,hd,jnj,smg
Date,,,,,,,,,,,
2006-11-29,-0.017327,-0.000109,0.004413,0.005115,0.010745,-0.003918,0.006486,-0.009958,0.014728,0.003481,0.001213
2006-11-30,-0.006262,-0.001526,0.012471,0.005089,-0.004058,-0.008675,-0.019587,0.000330,0.009260,-0.004390,-0.000606
2006-12-01,0.004075,-0.003716,-0.005669,-0.033552,0.011456,-0.021616,-0.035798,-0.008306,0.025996,0.000910,0.000202
2006-12-04,0.020128,-0.002192,0.021418,0.020780,0.000335,0.027460,0.013575,0.008388,0.005885,0.004688,0.003833
2006-12-05,0.016419,0.001645,-0.003860,-0.031334,0.012756,0.011275,-0.018141,0.004425,0.004327,-0.001661,0.001610
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,0.000469,0.000950,-0.002105,0.062979,-0.013571,0.005742,-0.020001,-0.004601,0.006638,-0.003489,0.000845
2019-12-26,0.001757,0.019646,-0.004341,0.016584,-0.009292,0.021414,0.039609,0.013329,0.000679,-0.000686,-0.000188
2019-12-27,-0.000351,-0.000380,-0.001647,-0.022454,0.000667,0.003446,-0.039609,-0.005763,-0.003856,-0.000549,0.001500


### Calculate Mean Returns

In [83]:
# Calculating Mean Returns

df_returns_mean = pd.DataFrame([{col: ((df_returns[col].mean() * 252)) for col in df_returns.columns}])

df_returns_mean

,a,aapl,abc,ag,ba,coke,ford,googl,hd,jnj,smg
0,0.105182,0.24846,0.117041,0.087624,0.124617,0.127026,-0.117682,0.129958,0.161476,0.090385,0.095211


### Calculate Risk

In [84]:
# Calculating Covariance Matrix
df_covariance_matrix = df_returns.cov() * 252

### Run Monte Carlo

In [85]:
risk_free_rate = 0

df_final = pd.DataFrame(columns=tickers)

# Randomly Assign Allocation
for iteration in range(iterations):
    new_df = {}
    new_df_list = []

    for ticker in tickers:
        new_df[ticker] = randint(0,100)
    new_df_list.append(new_df)

    allocation = pd.DataFrame(new_df_list)
    allocation_sum = allocation.sum(axis=1)
    
    for ticker in tickers:
        allocation[ticker] = allocation[ticker].apply(lambda x: x/allocation_sum)

    expected_return = (df_returns_mean * allocation).sum().sum()
    portfolio_risk = np.sqrt(reduce(np.dot, [allocation, df_covariance_matrix, allocation.T]))

    allocation["expected_return"] = expected_return
    allocation["portfolio_risk"] = portfolio_risk
    allocation["sharp_ratio"] = (expected_return - risk_free_rate) / portfolio_risk

    df_final = pd.concat([df_final, allocation])

df_final.sort_values("sharp_ratio", ascending=False)


,coke,googl,hd,ford,smg,jnj,ag,aapl,a,abc,ba,expected_return,portfolio_risk,sharp_ratio
0,0.047091,0.085873,0.210526,0.005540,0.105263,0.013850,0.008310,0.213296,0.113573,0.177285,0.019391,0.150594,0.179265,0.840065
0,0.032389,0.174089,0.168016,0.012146,0.026316,0.105263,0.048583,0.157895,0.022267,0.188259,0.064777,0.140395,0.175712,0.799010
0,0.230404,0.002375,0.166271,0.000000,0.064133,0.014252,0.057007,0.237530,0.002375,0.095012,0.130641,0.155481,0.196065,0.793007
0,0.153846,0.000000,0.088319,0.008547,0.131054,0.022792,0.048433,0.284900,0.028490,0.108262,0.125356,0.153655,0.193946,0.792257
0,0.084507,0.089202,0.194836,0.030516,0.002347,0.103286,0.037559,0.204225,0.077465,0.077465,0.098592,0.143290,0.182709,0.784250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.151057,0.120846,0.024169,0.229607,0.045317,0.060423,0.280967,0.006042,0.042296,0.024169,0.015106,0.056832,0.305100,0.186273
0,0.129577,0.070423,0.073239,0.270423,0.050704,0.047887,0.230986,0.033803,0.022535,0.008451,0.061972,0.054491,0.294882,0.184787
0,0.017632,0.231738,0.017632,0.244332,0.057935,0.002519,0.249370,0.007557,0.156171,0.007557,0.007557,0.054174,0.296893,0.182469
0,0.000000,0.004016,0.120482,0.325301,0.120482,0.044177,0.136546,0.072289,0.108434,0.020080,0.048193,0.046845,0.281205,0.166588
